In [7]:
# ========================================
# 1. 필요한 라이브러리 임포트
# ========================================
# Autogen: Microsoft에서 개발한 멀티 에이전트 AI 프레임워크
# 여러 AI 에이전트가 협업하여 복잡한 작업을 수행할 수 있게 해줍니다

# SelectorGroupChat: 여러 에이전트가 참여하는 그룹 채팅 시스템
# 상황에 따라 적절한 에이전트를 자동으로 선택하여 작업을 진행합니다
from autogen_agentchat.teams import SelectorGroupChat

# AssistantAgent: LLM(대규모 언어 모델) 기반의 AI 에이전트
# UserProxyAgent: 사람을 대리하는 에이전트 (사람의 입력을 받거나 승인을 요청)
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent

# OpenAI의 GPT 모델을 사용하기 위한 클라이언트
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 종료 조건: 에이전트 팀의 작업을 언제 끝낼지 정하는 조건들
# MaxMessageTermination: 최대 메시지 수에 도달하면 종료
# TextMentionTermination: 특정 텍스트가 나오면 종료
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

# Console: 에이전트 간 대화를 콘솔에 출력하는 UI
from autogen_agentchat.ui import Console

# 사용자 정의 도구들 (별도 파일에 정의되어 있음)
# web_search_tool: 웹 검색 기능
# save_report_to_md: 보고서를 마크다운 파일로 저장
from tools import web_search_tool, save_report_to_md


In [8]:
# ========================================
# 0. 모듈 자동 리로드 설정 (개발용)
# ========================================
# Jupyter 노트북에서 외부 모듈(tools.py)을 수정할 때마다 자동으로 리로드
# 이렇게 하면 커널을 재시작하지 않아도 코드 변경사항이 반영됩니다

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# ========================================
# 2. LLM 모델 클라이언트 설정
# ========================================
# 모든 에이전트가 사용할 언어 모델을 설정합니다
# gpt-4o-mini: OpenAI의 경량화된 GPT-4 모델 (빠르고 비용 효율적)
# 
# 참고: 환경변수에 OPENAI_API_KEY가 설정되어 있어야 합니다
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

In [10]:
# ========================================
# 3. AI 에이전트 정의
# ========================================
# 각 에이전트는 특정 역할을 수행하는 전문가입니다
# 에이전트 구성 요소:
# - name: 에이전트 이름 (고유 식별자)
# - description: 에이전트의 역할 설명 (다른 에이전트나 시스템이 이해하기 위함)
# - model_client: 사용할 LLM 모델
# - system_message: 에이전트의 행동 방식을 정의하는 시스템 프롬프트
# - tools: 에이전트가 사용할 수 있는 도구들 (선택적)

# ------------------------------
# 3.1 Research Planner (연구 기획자)
# ------------------------------
# 역할: 복잡한 질문을 구체적인 리서치 계획으로 분해
# 예: "핵에너지 최신 동향" → 구체적인 검색 쿼리 3-5개 생성
research_planner = AssistantAgent(
    "research_planner",
    description="A strategic research coordinator that breaks down complex questions into research subtasks",
    model_client=model_client,
    system_message="""You are a research planning specialist. Your job is to create a focused research plan.

For each research question, create a FOCUSED research plan with:

1. **Core Topics**: 2-3 main areas to investigate
2. **Search Queries**: Create 3-5 specific search queries covering:
   - Latest developments and news
   - Key statistics or data
   - Expert analysis or studies
   - Future outlook

Keep the plan focused and achievable. Quality over quantity.""",
)

# ------------------------------
# 3.2 Research Agent (리서치 실행자)
# ------------------------------
# 역할: 실제 웹 검색을 수행하고 정보를 수집
# 도구: web_search_tool을 사용하여 인터넷 검색
# 핵심: 계획에 따라 3-5개의 검색을 실행하고 주요 정보를 추출
research_agent = AssistantAgent(
    "research_agent",
    description="A web research specialist that searches and extracts content",
    tools=[web_search_tool],  # 이 에이전트만 웹 검색 도구를 사용할 수 있음
    model_client=model_client,
    system_message="""You are a web research specialist. Your job is to conduct focused searches based on the research plan.

RESEARCH STRATEGY:
1. **Execute 3-5 searches** from the research plan
2. **Extract key information** from the results:
   - Main facts and statistics
   - Recent developments
   - Expert opinions
   - Important context

3. **Quality focus**:
   - Prioritize authoritative sources
   - Look for recent information (within 2 years)
   - Note diverse perspectives

After completing the searches from the plan, summarize what you found. Your goal is to gather 5-10 quality sources.""",
)

# ------------------------------
# 3.3 Research Analyst (분석가)
# ------------------------------
# 역할: 수집된 정보를 바탕으로 체계적인 보고서 작성
# 출력: 요약, 배경, 분석, 전망, 출처가 포함된 완전한 보고서
# 완료 신호: "REPORT_COMPLETE"로 작업 완료를 표시
research_analyst = AssistantAgent(
    "research_analyst",
    description="An expert analyst that creates research reports",
    model_client=model_client,
    system_message="""You are a research analyst. Create a comprehensive report from the gathered research.

CREATE A RESEARCH REPORT with:

## Executive Summary
- Key findings and conclusions
- Main insights

## Background & Current State
- Current landscape
- Recent developments
- Key statistics and data

## Analysis & Insights
- Main trends
- Different perspectives
- Expert opinions

## Future Outlook
- Emerging trends
- Predictions
- Implications

## Sources
- List all sources used

Write a clear, well-structured report based on the research gathered. End with "REPORT_COMPLETE" when finished.""",
)

# ------------------------------
# 3.4 Quality Reviewer (품질 검토자)
# ------------------------------
# 역할: 보고서의 완성도를 확인하고 파일로 저장
# 도구: save_report_to_md를 사용하여 보고서를 마크다운 파일로 저장
# 확인 사항: 보고서 완성도, 출처의 신뢰성, 답변의 완성도
quality_reviewer = AssistantAgent(
    "quality_reviewer",
    description="A quality assurance specialist that evaluates research completeness and accuracy",
    tools=[save_report_to_md],  # 보고서 저장 도구
    model_client=model_client,
    system_message="""You are a quality reviewer. Your job is to check if the research analyst has produced a complete research report.

Look for:
- A comprehensive research report from the research analyst that ends with "REPORT_COMPLETE"
- The research question is fully answered
- Sources are cited and reliable
- The report includes summary, key information, analysis, and sources

When you see a complete research report that ends with "REPORT_COMPLETE":
1. First, use the save_report_to_md tool to save the report to report.md
2. Then say: "The research is complete. The report has been saved to report.md. Please review the report and let me know if you approve it or need additional research."

If the research analyst has NOT yet created a complete report, tell them to create one now.""",
)

# ------------------------------
# 3.5 Research Enhancer (연구 개선 제안자)
# ------------------------------
# 역할: 수집된 정보의 중요한 빈틈을 찾아내고 추가 조사 제안
# 주의: 사소한 부족함이 아닌 '중요한' 빈틈만 지적 (과도한 완벽주의 방지)
# 목표: 적절한 품질과 효율성의 균형
research_enhancer = AssistantAgent(
    "research_enhancer",
    description="A specialist that identifies critical gaps only",
    model_client=model_client,
    system_message="""You are a research enhancement specialist. Your job is to identify ONLY CRITICAL gaps.

Review the research and ONLY suggest additional searches if there are MAJOR gaps like:
- Completely missing recent developments (last 6 months)
- No statistics or data at all
- Missing a crucial perspective that was specifically asked for

If the research covers the basics reasonably well, say: "The research is sufficient to proceed with the report."

Only suggest 1-2 additional searches if absolutely necessary. We prioritize getting a good report done rather than perfect coverage.""",
)

# ------------------------------
# 3.6 User Proxy (사용자 대리인)
# ------------------------------
# 역할: 실제 사용자의 입력과 피드백을 받음
# 중요: 이 에이전트가 활성화되면 사용자에게 입력을 요청함
# 사용 시점: 최종 보고서 승인 또는 추가 조사 요청
user_proxy = UserProxyAgent(
    "user_proxy",
    description="Human reviewer who can request additional research or approve final results",
    input_func=input,  # 파이썬의 input() 함수를 사용하여 사용자 입력 받음
)

In [11]:
# ========================================
# 4. Selector 프롬프트 정의
# ========================================
# SelectorGroupChat에서 사용할 에이전트 선택 로직
# 이 프롬프트는 대화 흐름을 보고 다음에 어떤 에이전트가 작업할지 결정합니다
# 
# 핵심 개념: 워크플로우 자동화
# - 각 단계에서 적절한 전문가(에이전트)를 자동으로 선택
# - 순차적 작업 흐름 보장 (기획 → 조사 → 개선 → 분석 → 검토 → 사용자 승인)

selector_prompt = """
Choose the best agent for the current task based on the conversation history:

{roles}

Current conversation:
{history}

Available agents:
- research_planner: Plan the research approach (ONLY at the start)
- research_agent: Search for and extract content from web sources (after planning)
- research_enhancer: Identify CRITICAL gaps only (use sparingly)
- research_analyst: Write the final research report
- quality_reviewer: Check if a complete report exists
- user_proxy: Ask the human for feedback

WORKFLOW:
1. If no planning done yet → select research_planner
2. If planning done but no research → select research_agent  
3. After research_agent completes initial searches → select research_enhancer ONCE
4. If enhancer says "sufficient to proceed" → select research_analyst
5. If enhancer suggests critical searches → select research_agent ONCE more then research_analyst
6. If research_analyst said "REPORT_COMPLETE" → select quality_reviewer
7. If quality_reviewer asked for user feedback → select user_proxy

IMPORTANT: After research_agent has searched 2 times maximum, proceed to research_analyst regardless.

Pick the agent that should work next based on this workflow."""

# 학습 포인트:
# - 이 프롬프트는 멀티 에이전트 시스템의 '교통 정리' 역할
# - LLM이 이 프롬프트를 읽고 상황에 맞는 에이전트를 선택
# - {roles}와 {history}는 실행 시 실제 값으로 자동 치환됨
# - 워크플로우가 명확하게 정의되어 있어 작업이 순차적으로 진행됨

In [12]:
# ========================================
# 5. 종료 조건 설정 및 팀 구성
# ========================================

# ------------------------------
# 5.1 종료 조건 정의
# ------------------------------
# 에이전트 팀이 언제 작업을 멈출지 결정하는 조건들

# TextMentionTermination: 특정 텍스트가 대화에 나타나면 종료
# "APPROVED"라는 단어가 나오면 작업 완료로 판단
text_termination = TextMentionTermination("APPROVED")

# MaxMessageTermination: 최대 메시지 수 제한
# 무한 루프 방지 - 50개 메시지 후 자동 종료
max_message_termination = MaxMessageTermination(max_messages=50)

# 종료 조건 결합: OR 연산자(|) 사용
# "APPROVED"가 나오거나 50개 메시지에 도달하면 종료
termination_condition = text_termination | max_message_termination

# ------------------------------
# 5.2 SelectorGroupChat 팀 구성
# ------------------------------
# 여러 에이전트가 협업하는 그룹 채팅 시스템 생성
team = SelectorGroupChat(
    # participants: 팀에 참여하는 모든 에이전트 목록
    # 순서는 중요하지 않음 - selector_prompt가 선택을 결정
    participants=[
        research_agent,      # 웹 검색 실행자
        research_analyst,    # 보고서 작성자
        research_enhancer,   # 품질 개선 제안자
        research_planner,    # 연구 기획자
        quality_reviewer,    # 최종 검토자
        user_proxy,          # 사용자 대리인
    ],
    
    # selector_prompt: 다음 에이전트를 선택하는 로직
    selector_prompt=selector_prompt,
    
    # model_client: 에이전트 선택을 위한 LLM
    # 이 모델이 selector_prompt를 읽고 다음 에이전트 결정
    model_client=model_client,
    
    # allow_repeated_speaker: 같은 에이전트의 연속 발언 허용 여부
    # 주석 처리됨 (기본값: False) - 다양한 에이전트가 번갈아 작업
    # allow_repeated_speaker=True,
    
    # termination_condition: 팀 작업 종료 조건
    termination_condition=termination_condition,
)

# 학습 포인트:
# - SelectorGroupChat은 Autogen의 핵심 개념
# - 각 에이전트가 자신의 전문 분야에서만 작업
# - selector가 자동으로 적절한 에이전트를 선택하여 효율적인 협업
# - 종료 조건으로 무한 루프 방지 및 명확한 완료 시점 정의

In [ ]:
# ========================================
# 6. 팀 실행
# ========================================
# 설정한 에이전트 팀을 실제로 작동시키는 부분

# Console: 에이전트 간 대화를 콘솔에 실시간으로 출력
# team.run_stream(): 팀을 스트리밍 모드로 실행
#   - 각 에이전트의 응답을 실시간으로 볼 수 있음
#   - task 매개변수: 사용자가 요청하는 초기 작업

await Console(
    team.run_stream(task="Research about the new development in Nuclear Energy"),
)

# 실행 흐름 예시:
# 1. research_planner: 핵에너지 관련 검색 쿼리 3-5개 생성
# 2. research_agent: 웹 검색 수행 및 정보 수집
# 3. research_enhancer: 추가 조사가 필요한지 판단
# 4. research_analyst: 수집된 정보로 보고서 작성
# 5. quality_reviewer: 보고서를 report.md에 저장하고 사용자에게 검토 요청
# 6. user_proxy: 사용자 입력 대기 ("APPROVED" 입력 시 종료)

# 학습 포인트:
# - await: 비동기 함수이므로 await 사용 (Jupyter 노트북은 비동기 지원)
# - task 매개변수를 바꾸면 다른 주제로 리서치 가능
# - Console UI가 각 에이전트의 사고 과정과 대화를 보여줌
# - 전체 과정이 자동화되어 있지만 user_proxy를 통해 사람의 개입 가능

---------- TextMessage (user) ----------
Research about the new development in Nuclear Energy
---------- TextMessage (research_planner) ----------
### Research Plan: New Development in Nuclear Energy

#### 1. Core Topics:
   - **Innovative Reactor Designs**: Focus on small modular reactors (SMRs) and advanced reactor technologies.
   - **Regulatory Environment**: Investigate changes in regulations and policies affecting nuclear energy development globally.
   - **Nuclear Energy's Role in Climate Goals**: Explore how nuclear energy is being integrated into national and international climate strategies.

#### 2. Search Queries:
- **Latest Developments and News**:
   - "latest advancements in small modular reactors 2023"
   - "recent nuclear energy projects 2023 news"

- **Key Statistics or Data**:
   - "global nuclear energy capacity statistics 2023"
   - "trends in nuclear energy adoption by country 2023"

- **Expert Analysis or Studies**:
   - "expert opinions on advanced nuclear react